In [1]:
import os
os.chdir("./../.")

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import json
import dotenv

In [4]:
dotenv.load_dotenv()

True

In [5]:

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [6]:
from src.utils_stock import get_top_nasdaq_stock_data

In [7]:
# parameter
NUM_DAYS = 5

In [8]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [9]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

### Data Acquisition Agent

In [10]:
@tool
def get_top_nasdaq_performance_stock_data() -> str:
    """Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.
    """
    result = get_top_nasdaq_stock_data(NUM_DAYS)
    return json.dumps(result)

In [11]:
# Research agent and node
data_acquisition_agent = create_react_agent(
    llm,
    tools=[get_top_nasdaq_performance_stock_data],
    prompt=get_system_prompt(
        "You can only do data acquisition to the day's top NASDAQ gainer info using get_top_nasdaq_performance_stock_data tool."
    ),
)

### Python Data Analysis Agent

In [12]:
from langchain_experimental.utilities import PythonREPL

In [13]:
from typing import Annotated


repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to do comprehensive stock performance analysis"],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [14]:
python_data_analysis_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=get_system_prompt(
        """
        You are an agent designed to write and execute python code for comprehensive stock analysis given data. 
        The data include the stock symbol, percentage increase, sample data in markdown format, pandas dataframe info, and csv data path for python code analysis. 
        You have access to a python REPL, which you can use to execute python code.
        If you get an error, debug your code and try again.
        You're expected to read the data using pandas to perform statistical analysis such as last 5 trading days (e.g., calculate average change, determine a simple trend)
        and calculating volatility, comparing performance to a market index like S&P 500
"""
    ),
)

### Define Graph

In [15]:
from typing import Literal
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, BaseMessage

In [16]:
# def get_next_node(last_message: BaseMessage, goto: str):
#     if "FINAL ANSWER" in last_message.content:
#         # Any agent decided the work is done
#         return END
#     return goto

In [17]:
def data_acquisition_node(
    state: MessagesState,
) -> Command[Literal["python_data_analyst"]]:   # Go to python_data_analyst node
    result = data_acquisition_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="data_acquisition_specialist"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto="python_data_analyst",
    )

In [18]:
def data_analysis_node(
    state: MessagesState,
) -> Command[Literal[END]]:  # type: ignore
    result = python_data_analysis_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="python_data_analyst"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto=END,
    )

In [19]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(MessagesState)
workflow.add_node("data_acquisition_specialist", data_acquisition_node)
workflow.add_node("python_data_analyst", data_analysis_node)

workflow.add_edge(START, "data_acquisition_specialist")
# workflow.add_edge("data_acquisition_specialist", "python_data_analyst")
graph = workflow.compile()

In [20]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [21]:
events = graph.stream(
    {
        "messages": [
            (
                "user",
                "First, get the top performance stock data "
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
)
for s in events:
    print(s)
    print("----")

100%|██████████| 101/101 [00:02<00:00, 35.77it/s]


{'data_acquisition_specialist': {'messages': [HumanMessage(content='First, get the top performance stock data ', additional_kwargs={}, response_metadata={}, id='e26f9471-6601-4321-89bb-97e03f1c1550'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_7fa32033-ef71-418d-80b9-7efb7de84a96', 'function': {'arguments': '{}', 'name': 'get_top_nasdaq_performance_stock_data'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 237, 'total_tokens': 260, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '73f647c6-eb98-4c0f-beca-8d19535103b3', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8196bbf3-eeda-4cba-85a6-cb819f2cf10d-0', tool_calls=[{'name': 'get_top_n

Python REPL can execute arbitrary code. Use with caution.


{'python_data_analyst': {'messages': [HumanMessage(content='First, get the top performance stock data ', additional_kwargs={}, response_metadata={}, id='e26f9471-6601-4321-89bb-97e03f1c1550'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_7fa32033-ef71-418d-80b9-7efb7de84a96', 'function': {'arguments': '{}', 'name': 'get_top_nasdaq_performance_stock_data'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 237, 'total_tokens': 260, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '73f647c6-eb98-4c0f-beca-8d19535103b3', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8196bbf3-eeda-4cba-85a6-cb819f2cf10d-0', tool_calls=[{'name': 'get_top_nasdaq_pe

In [22]:
from IPython.display import display, Markdown
display(Markdown(s['python_data_analyst']['messages'][-1].content))

### Analysis Results for MNST (Top NASDAQ-100 Gainer)

1. **Average Daily Change**: $0.02 (positive trend)
2. **Volatility**: $0.82 (standard deviation of daily changes)
3. **Last 5 Trading Days**:
   - **2025-05-05**: Close = $60.91
   - **2025-05-06**: Close = $60.02 (Change = -$0.89)
   - **2025-05-07**: Close = $60.56 (Change = +$0.54)
   - **2025-05-08**: Close = $60.14 (Change = -$0.42)
   - **2025-05-09**: Close = $61.00 (Change = +$0.86)

FINAL ANSWER: The stock MNST shows a positive trend with low average daily change ($0.02) and moderate volatility ($0.82). The last 5 days of trading data are provided above. Let me know if you'd like further analysis or comparison with a market index like the S&P 500.